# Inicialización

In [39]:
import logging

import numpy as np
import pandas as pd

import torch
import transformers

# Cargar datos

Carga los datos de texto del archivo 'imdb_reviews_small.tsv'.

Se trata de un archivo de valores separados por tabuladores (TSV), lo cual significa que cada uno de los campos está separado por tabuladores (en lugar de por comas como has visto en otros ejercicios de Practicum).

In [40]:
data = pd.read_csv("datasets/imdb_reviews_small.tsv", sep="\t")

# Tokenizador BERT

Crear el tokenizador BERT a partir de un modelo previamente entrenado que se llama 'bert-base-uncased' en transformadores. Puedes echar un vistazo a su descripción general [aquí](https://huggingface.co/transformers/pretrained_models.html).  Puedes encontrar más detalles [aquí](https://huggingface.co/bert-base-uncased).

In [41]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

Hay un ejemplo de cómo obtener tokens para un solo texto.

Puedes usarlo para procesar todos los datos que cargaste anteriormente. Como ya hay muchos textos, y es probable que los proceses en un bucle, las longitudes mínimas/máximas de los vectores se pueden calcular de dos formas: dentro de un bucle o después de un bucle.

En el último caso, los vectores de identificadores numéricos de tokens (ids) y máscaras de atención (attention_mask) se deben almacenar en dos listas separadas. Se pueden llamar ids_list y atencion_mask_list, respectivamente. El primer caso te permite evitar la creación de esas listas, a menos que desees utilizarlas con otra finalidad, por ejemplo, para propagarlas en un modelo BERT. No se requiere en este ejercicio, pero se requerirá en el proyecto.
Teniendo en cuenta lo anteriormente dicho, es posible que desees combinar ambas formas para calcular las longitudes mínimas/máximas de los vectores para tokens y máscaras de atención, así como conservar el resultado del tokenizador para su posterior procesamiento. Solo considera que no tiene mucho sentido mantener vectores de más de 512 elementos, ya que esta es la longitud máxima de vectores que BERT puede aceptar.

In [42]:
# textos a tokens
text = 'Es muy práctico utilizar transformadores'

# agregar este truco para suprimir las advertencias de salidas largas
# normalmente no es necesario, pero en este caso nos gustaría explorar
# ¿cuál es la longitud máxima de los ids para nuestro conjunto de reseñas?
# por lo tanto, no truncamos la salida (ids) a la max_length
# con los parámetros max_length=max_length y truncation=Truelogging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)
        
ids = tokenizer.encode(text.lower(), add_special_tokens=True)

# padding (agregar ceros al vector para hacer que su longitud sea igual a n)
n = 512
padded = np.array(ids[:n] + [0]*(n - len(ids)))

# crear la máscara de atención para distinguir los tokens que nos interesan
attention_mask = np.where(padded != 0, 1, 0)

In [43]:
print(ids)

[101, 9686, 14163, 2100, 10975, 28804, 2080, 21183, 18622, 9057, 10938, 26467, 2229, 102]


In [44]:
print(padded)

[  101  9686 14163  2100 10975 28804  2080 21183 18622  9057 10938 26467
  2229   102     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [45]:
print(attention_mask)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

Escribe tu código para tokenizar los datos de texto cargados.

In [46]:
def tokenize_with_bert(texts):
    
    ids_list = []
    attention_mask_list = []
    max_length = 512

    tokenized_texts = texts.apply(
        lambda x: 
        tokenizer.encode(
            x.lower(),
            add_special_tokens=True,
            truncation=True,
            max_length=max_length
        )
    )
    
    min_tokenized_text_length = min(len(ids) for ids in tokenized_texts)
    max_tokenized_text_length = max(len(ids) for ids in tokenized_texts)
    
    for ids in tokenized_texts:
        padded = np.array(ids[:max_length] + [0]*(max_length - len(ids)))
        ids_list.append(padded)
        attention_mask_list.append(np.where(padded != 0, 1, 0))
        
    print(f'La longitud mínima de los vectores: {min_tokenized_text_length}')
    print(f'La longitud máxima de los vectores:{max_tokenized_text_length}')        
        
    return ids_list, attention_mask_list

Ejecuta el tokenizador para todos los datos. Puede llevar algún tiempo ya que

In [47]:
ids_list, attention_mask_list = tokenize_with_bert(texts=data['review'])

La longitud mínima de los vectores: 16
La longitud máxima de los vectores:512
